<a href="https://colab.research.google.com/github/amuda005/randomfun/blob/master/ThresholdAUC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

In [0]:
#Converting images to tensor and normalizing them
generic_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,),(0.5,))])
#importing mnist data
train_set = torchvision.datasets.MNIST(root='./data', train=True, download=True,transform=generic_transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True,transform=generic_transform)

#extracting two classes - 4 & 8
idx = train_set.targets==4
idx +=train_set.targets==8
train_set.targets=np.multiply(train_set.targets[idx] > 4,1) #reindexing class 4 and 8 to 0 and 1 for compatibility with cross-entropy
train_set.data=train_set.data[idx]
print(f'size of training data with 4 and 8 classes: {len(train_set.targets)}')
idx=testset.targets==4
idx+=testset.targets==8
testset.targets=np.multiply(testset.targets[idx] > 4,1)
testset.data=testset.data[idx]

#splitting data for checking generalizability of model
train_set, true_set = torch.utils.data.random_split(train_set, [9693, 2000])
trainloader=torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
testloader=torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)
truesetloader=torch.utils.data.DataLoader(true_set, batch_size=32, shuffle=False) 

In [0]:
print(f'size of train dataset: {len(train_set.indices)}')
print(f'size of test dataset: {len(testset.targets)}')
print(f'size of true perf dataset: {len(true_set.indices)}')

In [0]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 20, 3, bias= True)
    self.fc1 = nn.Linear(20 * 13 * 13, 128)
    self.fc2 = nn.Linear(128, 2, bias= True) #adjusting for two classes

  def forward(self, x):
    x = F.relu(F.max_pool2d(self.conv1(x), (2, 2)))
    x = x.view(-1, self.num_flat_features(x))
    x = F.relu(self.fc1(x))
    x = F.softmax(self.fc2(x),dim=1)
    return x

  def num_flat_features(self, x):
    size = x.size()[1:]  # all dimensions except the batch dimension
    num_features = 1
    for s in size:
        num_features *= s
    return num_features

In [0]:
def predict(testnet,loader,k=0):
  total=0
  correct=0
  preds = []
  actual = []
  for i,data in enumerate(loader):
    inputs,labels = data
    actual.extend(labels)
    outputs = testnet(inputs)
    _, predicted = torch.max(outputs.data, 1)
    preds.extend(predicted)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    if(k !=0 and i>=k): 
      break
  accuracy = 100 * correct / total
  return preds,actual,accuracy

In [0]:
# run the model on test data and return auc score
def naive_auc(model,loader,k=0):
  preds,actual,test_acc = predict(model,loader,k)  
  test_auc = roc_auc_score(actual,preds)
  return test_auc

In [0]:
no_of_rounds=10
n_epochs = 10
delta = 0.0005 #check for model convergence

In [0]:
test_aucs = []
true_aucs = []
train_aucs = []
best_lrs = []
lr=[0.9,0.5,0.1,0.09,0.05,0.01,0.009,0.005,0.001,0.0001]
for r in range(no_of_rounds):#Rounds of adaptive learning
  #Eliminating worst learning rate each round
  max_test_auc= 0
  max_train_auc = 0
  worst_lr=0
  best_lr=0
  worst_test_auc=1.0
  print(f'Round {r} : ')
  k = 4800/32 + (r+1)*(500/32)
  for l in lr:
    net = Net()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=l)
    grad = 0
    pre_grad = -1
    pre_loss = -10
    for epoch in range(n_epochs):
      if(abs(grad-pre_grad) < delta):
          print(f'    Model converged at epoch={epoch+1}')
          break
      actual=[]
      preds=[]
      for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss=criterion(outputs, labels)
        pre_grad = grad
        grad = loss-pre_loss
        pre_loss = loss
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        preds.extend(predicted.tolist())
        actual.extend(labels.tolist())
        if(i>=k):
          break
    with torch.no_grad():
      train_auc = roc_auc_score(actual,preds)
      t = naive_auc(net,testloader)
      print(f'    Finished Training after epoch {epoch+1} for learning rate: {l}, batch: {i}, Training AUROC: {train_auc} Testing AUROC: {t}')    
      if(worst_test_auc > t):
        worst_test_auc = t
        worst_lr = l
      if(max_test_auc < t):
        max_net = net
        best_lr=l
        max_test_auc = t
        max_train_auc = train_auc
  if(lr.count(worst_lr)>0):
     lr.remove(worst_lr)
  train_aucs.append(max_train_auc)
  test_aucs.append(max_test_auc)
  true_aucs.append(naive_auc(max_net,truesetloader))
  best_lrs.append(best_lr)
  print(f' Worst Learning rate:  {worst_lr} with test auc: {worst_test_auc}')    
  print(f' Best Testing AUROC:  {max_test_auc} is for learning rate: {best_lr}')    
  print(f' True AUROC:  {true_aucs}')    

In [0]:
print(f'train performance: {train_aucs}')
print(f'test performance: {test_aucs}')
print(f'true performance: {true_aucs}')

In [0]:
plt.title("Naive AUC")
plt.xlim((0,10))
plt.ylim((0.99,1))
plt.plot(train_aucs,color='green',label="Training Performance")
plt.plot(test_aucs,color="red",label="Testing Performance")
plt.plot(true_aucs,color="blue",label="True Performance")
plt.legend(loc='lower left')
plt.show()

In [0]:
#Set threshold, budget
#Function that takes model parameters and returns auc score according to thresholdout mechanism of c.dwork et.al
B=7
def thresholdout_auc(model,trainloader,k=0, T=0.02, sigma =0.03):
#get the training data AUC by running 5-fold cross validation on the training data
#check the difference between training and test data is greater than T(Threshold+noise) + noise
  eps = np.random.laplace(sigma, 1)
  neta = np.random.laplace(sigma*4,1)
  gamma = np.random.laplace(2*sigma,1)
  T1=T+gamma
  train_auc=naive_auc(model,trainloader,k)
  test_auc = naive_auc(model,testloader,0)
  global B
  if(np.abs(train_auc-test_auc)>(T1+neta) and B>0):
    B=B-1
    T1=T+gamma
    test_auc=max(0,test_auc+eps)
    test_auc=min(1,test_auc)
    return test_auc
  #if check is true return test auc+noise
#decrease your budget of adaptive queries(#adaptive queries)
#else return training auc
  return train_auc

In [0]:
test_thresh_aucs = []
true_thresh_aucs = []
train_thresh_aucs = []
best_thresh_lrs = []
lr=[0.9,0.5,0.1,0.09,0.05,0.01,0.009,0.005,0.001,0.0001]
B=7
for r in range(no_of_rounds):#Rounds of adaptive learning
  #Eliminating worst learning rate each round
  max_test_auc= 0
  max_train_auc = 0
  worst_lr=0
  best_lr=0
  worst_test_auc=1.0
  print(f'Round {r} : ')
  k = 4800/32 + (r+1)*(500/32)
  for l in lr:
    net = Net()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=l)
    grad = 0
    pre_grad = -1
    pre_loss = -10
    for epoch in range(n_epochs):
      if( abs(grad-pre_grad) < delta ):
          print(f' Model converged at epoch={epoch+1}')
          break
      actual=[]
      preds=[]
      for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss=criterion(outputs, labels)
        pre_grad = grad
        grad = loss-pre_loss
        pre_loss = loss
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        preds.extend(predicted.tolist())
        actual.extend(labels.tolist())
        if(i>=k):
          break
    with torch.no_grad():
      train_auc = roc_auc_score(actual,preds)
      t = thresholdout_auc(net,trainloader)
      print(f' Finished Training after epoch {epoch+1} for learning rate: {l}, batch: {i}, Training AUROC:  {train_auc}, Testing AUROC:  {t}')    
      if(worst_test_auc > t):
        worst_test_auc = t
        worst_lr = l
      if(max_test_auc < t):
        max_net = net
        best_lr=l
        max_test_auc = t
        max_train_auc = train_auc
  if(lr.count(worst_lr)>0):
     lr.remove(worst_lr)
  train_thresh_aucs.append(max_train_auc)
  test_thresh_aucs.append(max_test_auc)
  true_thresh_aucs.append(naive_auc(max_net,truesetloader))
  best_thresh_lrs.append(best_lr)
  print(f' Worst Learning rate:  {worst_lr} with test auc: {worst_test_auc}')    
  print(f' Best Testing AUROC:  {max_test_auc} is for learning rate: {best_lr}')    
  print(f' True AUROC:  {true_thresh_aucs}')    

 Finished Training after epoch 10 for learning rate: 0.1, batch: 182, Training AUROC:  0.9964129654650161, Testing AUROC:  0.9974209785529639
 Finished Training after epoch 10 for learning rate: 0.09, batch: 182, Training AUROC:  0.9965849311457359, Testing AUROC:  0.9977307676588092
 Finished Training after epoch 10 for learning rate: 0.05, batch: 182, Training AUROC:  0.9953802653596013, Testing AUROC:  0.996286043077992
 Model converged at epoch=6
 Finished Training after epoch 6 for learning rate: 0.01, batch: 182, Training AUROC:  0.9873305644096719, Testing AUROC:  0.9905081665287273
 Model converged at epoch=7
 Finished Training after epoch 7 for learning rate: 0.009, batch: 182, Training AUROC:  0.9877310246770077, Testing AUROC:  0.9877233214807549


In [0]:
print(f'train performance: {train_thresh_aucs}')
print(f'test performance: {test_thresh_aucs}')
print(f'true performance: {true_thresh_aucs}')

In [0]:
plt.title("Threshold AUC")
plt.xlim((0,10))
plt.ylim((0.97,1.01))
plt.plot(train_thresh_aucs,color='green',label="Train Perf")
plt.plot(test_thresh_aucs,color="red",label="Test Perf")
plt.plot(true_thresh_aucs,color="blue",label="True Perf")
plt.legend(loc='lower left')
plt.show()